# Run Freesurfer Recon-All using nipype

In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import moss
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.display import Image

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
%matplotlib inline

In [3]:
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'new_subs.txt'
subjects = np.loadtxt(subj_file,'string')
subjects_dir = home_dir + 'data/freesurfer/'
subjects = [str(subjects)]
subjects = ['asu11','asu12']
subjects

['asu11', 'asu12']

In [3]:
#initialize workflow
wf = Workflow('freesurfer_recon')
wf.base_dir = abspath(home_dir)

In [4]:
#get data using datagrabber
##set up data grabber
ds = Node(DataGrabber(infields=['subject_id','task_id','run_id'],outfields =['func','anat']),
         name = 'datasource')
ds.inputs.base_directory = abspath(home_dir + '/data/')
ds.inputs.template = '*'
ds.inputs.template_args = {'anat':[['subject_id']]}
ds.inputs.field_template = {'anat': '%s/anat/st1_0000.nii.gz'}
ds.inputs.sort_filelist = True

ds.inputs.subject_id = subjects
ds.inputs.task_id = '*'
ds.inputs.run_id = '*'
print ds.run().outputs


anat = ['/data/home/iballard/mag/data/asu11/anat/st1_0000.nii.gz', '/data/home/iballard/mag/data/asu12/anat/st1_0000.nii.gz']



In [5]:
#set up recon node
recon_all = MapNode(interface = ReconAll(), name = 'recon_all',iterfield = ['subject_id','T1_files'])
recon_all.inputs.subject_id  = subjects
recon_all.inputs.subjects_dir = subjects_dir
# recon_all.inputs.directive  = 'autorecon3'
# recon_all.inputs.args = '-noskullstrip'
wf.connect(ds,'anat',recon_all,'T1_files')
print recon_all.inputs


T1_files = <undefined>
T2_file = <undefined>
args = <undefined>
directive = all
environ = {}
flags = <undefined>
hemi = <undefined>
ignore_exception = False
openmp = <undefined>
subject_id = ['asu11', 'asu12']
subjects_dir = /data/home/iballard/mag/data/freesurfer/
terminal_output = stream



In [6]:
wf.run(plugin = 'MultiProc')

/home/iballard/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/home/iballard/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [4]:
##fix directory structure. For some reason ipython insists on making a new directory for freesurfer outputs,
#but lyman wants them in the subject folder
for n,sub in enumerate(subjects):
    print sub
    if True:
        freesurfer_dir = home_dir + '/data/freesurfer/' + sub
        if os.path.exists(freesurfer_dir):
            sub_dir = home_dir + '/data/' + sub +'/'
            cmd = 'mv ' + freesurfer_dir + '/* ' + sub_dir
            print cmd
            os.system(cmd)
            os.rmdir(freesurfer_dir)


asu11
mv /data/home/iballard/mag//data/freesurfer/asu11/* /data/home/iballard/mag//data/asu11/
asu12
mv /data/home/iballard/mag//data/freesurfer/asu12/* /data/home/iballard/mag//data/asu12/
